# Implementing Iteration

## Agenda

1. Review: Iteration
2. Details: *iterables*, *iterators*, `iter`, and `next`
3. Implementing iterators with classes
4. Implementing iterators with *generators* and `yield`

## 1. Iteration

*Iteration* simply refers to the process of accessing — one by one — the items stored in some container. The order of the items, and whether or not the iteration is comprehensive, depends on the container.

In Python, we typically perform iteration using the `for` loop.

In [1]:
for i in range(10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [2]:
for c in 'CS 331':
    print(c)

C
S
 
3
3
1


In [3]:
for e in [3.14, 'hello', True]:
  print(e)

3.14
hello
True


In [4]:
for n in {5, 6, 7, 5, 3, 0, 9}:
  print(n)

0
3
5
6
7
9


In [7]:
for k,v in {'a': 'aardvark', 'b': 'bear', 'c': 'cat'}.items():
    print(f'{k} => {v}')

a => aardvark
b => bear
c => cat


## 2. *iterables*, *iterators*, `iter`, and `next`

We can iterate over anything that is *iterable*. Intuitively, if something can be used as the source of items in a `for` loop, it is iterable.

But how does a `for` loop really work?

In [8]:
l = ['iteration', 'is', 'really', 'very', 'convenient']

In [9]:
it = iter(l)

the global function iter gets you back an iterator for an argument that is iterable

In [10]:
type(l), type(it)

(list, list_iterator)

In [16]:
next(it)

StopIteration: 

When you call next on an iterator, it gives you back the next element from the underlying iterable

In [18]:
# putting it all together
it = iter(l)
while True:
    try:
        x = next(it)
        print(x)
    except StopIteration:
        break

iteration
is
really
very
convenient


In [20]:
# the above function is semantically the same as the following:
for i in l:
    print(i)

iteration
is
really
very
convenient


In [21]:
# iter & next are based on the __iter__ and __next__ special methods
# in actuality, this is what goes on behind the scenes when the iter and next methods are called
it = l.__iter__()
while True:
    try:
        x = it.__next__()
        print(x)
    except StopIteration:
        break

iteration
is
really
very
convenient


## 3. Implementing iterators with classes

In [22]:
class MyIterator:
    def __init__(self, max):
        self.max = max  # max value
        self.curr = 0   # current value
        
    # the following methods are required for iterator objects

    def __iter__(self):
        return self
    
    def __next__(self):
        if self.curr < self.max:
            ret = self.curr
            self.curr += 1
            return ret
        else:
            raise StopIteration

In [23]:
it = MyIterator(10)

In [34]:
next(it)

StopIteration: 

In [35]:
it = MyIterator(10)
while True:
    try:
        print(next(it))
    except StopIteration:
        break

0
1
2
3
4
5
6
7
8
9


In [36]:
it = MyIterator(10)
for i in it:
    print(i)

0
1
2
3
4
5
6
7
8
9


An iterator is a *one time use object*! I.e., once we've used it to iterate over elements we cannot typically reset or "rewind" iteration. 

In [37]:
it = MyIterator(10)

In [39]:
# try running this cell multiple times
for i in it:
  print(i)

Iterable objects that can be traversed repeatedly return fresh iterators for each traversal.

In [40]:
# what is happening each time we iterate over `l`, below?

l = list(range(10))
total = 0
for _ in range(10):
    for x in l:
        total += x
total

450

In [41]:
l = list(range(10))
total = 0
for _ in range(10):
    it = iter(l) # we obtain and "use up" a new iterator each loop!
    while True:
        try:
            total += next(it)
        except StopIteration:
            break
total

450

For a container type, we need to implement an `__iter__` method that returns an iterator.

In [55]:
import numpy as np

class ArrayListIterator:
    def __init__(self, array_list):
        self.array_list = array_list    # this is the underlying iterable
        self.curr_idx = 0   # this tracks our position in it

    def __iter__(self):
        return self

    def __next__(self):
        if self.curr_idx < self.array_list.size:
            ret = self.array_list.data[self.curr_idx]
            self.curr_idx += 1
            return ret
        else:
            raise StopIteration


class ArrayList:
    def __init__(self):
        self.data = np.empty(1, dtype=object)
        self.size = 0
        
    def append(self, val):
        if self.size == len(self.data):
            ndata = np.empty(2 * len(self.data), dtype=object)
            for i in range(len(self.data)):
                ndata[i] = self.data[i]
            self.data = ndata

        self.data[self.size] = val
        self.size += 1
        
    def __iter__(self):
        return ArrayListIterator(self) # create and return an iterator

In [56]:
l = ArrayList()
for x in range(10):
    l.append(2**x)

In [57]:
it = iter(l)

In [58]:
type(it)

__main__.ArrayListIterator

In [69]:
next(it)

StopIteration: 

In [71]:
for x in l:
    print(x)

1
2
4
8
16
32
64
128
256
512


## 4. Implementing iterators with generators

What's a "generator"?

In [72]:
l = [2**x for x in range(10)]
g = (2**x for x in range(10))

In [73]:
type(l), type(g)

(list, generator)

In [75]:
# try running this cell repeatedly
for x in l:
    print(x)

1
2
4
8
16
32
64
128
256
512


In [77]:
# try running this cell repeatedly
for x in g:
    print(x)

In [78]:
dir(g)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

Note: Generators implement the iterator API! (`__iter__` and `__next__`)

In [79]:
g = (2**x for x in range(10))

In [80]:
g is iter(g)

True

In the same way an iterator returns an iterator, a generator works the same way and returns an iterator

In [91]:
next(g)

StopIteration: 

In [92]:
%timeit -n 1000 [2**x for x in range(1000)]

595 µs ± 41.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [93]:
%timeit -n 1000 (2**x for x in range(1000))

The slowest run took 4.31 times longer than the fastest. This could mean that an intermediate result is being cached.
944 ns ± 442 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Generators are way more efficient to create than lists because generators don't actually contain all the values, only creating values as you need them, wheras lists create all the values

In [94]:
g = (2**x for x in range(100))

In [95]:
g[100]

TypeError: 'generator' object is not subscriptable

A generator object is not subscriptable, where you can't get things in a generator from their indexes (they don't keep the  values around)

In [97]:
# try running this cell repeatedly
sum(g)

0

A *generator expression* syntactically resembles a list comprehension, and is similar in that it evaluates to an iterable sequence of values. However, a generator does not represent a fully fleshed out collection of values; instead, values are returned only as they are required through the iteration API (i.e., `next`) --- we refer to this as *lazy evaluation*. 

This makes a generator more efficient than a list (since we don't need to keep all values in the sequence around), but generators can't replace lists in all scenarios (e.g., when we need to jump around in the sequence or revisit values).

### Creating generator functions: `yield`

In [98]:
def foo():
    yield

In [99]:
foo()

<generator object foo at 0x7605441149e0>

In [100]:
type(foo())

generator

In [101]:
def foo():
    print('hello!')
    yield
    print('goodbye!')

In [102]:
foo()

<generator object foo at 0x7605441142e0>

The presence of the yield statement changes the nature of the function, essentiall making it into a generator function

In [103]:
g = foo()

In [105]:
next(g)

goodbye!


StopIteration: 

Yield essentially pauses the excecution of a generator function, it remembers the state it left off on and continues from there

In [106]:
def foo():
    yield 1
    yield 2
    yield 3

In [107]:
g = foo()

In [111]:
next(g)

StopIteration: 

In [112]:
def countdown(n):
    for x in range(n, 0, -1):
        yield x
    yield 'Blast off!'

In [113]:
for x in countdown(5):
    print(x)

5
4
3
2
1
Blast off!


In [114]:
list(countdown(10))

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 'Blast off!']

A *generator function* is a function that contains one or more `yield` statements. When called, a generator function returns a generator object, which effectively allows us to incrementally execute the function using the iteration API. Each call to `next` on the generator will execute the function up to the next `yield` statement; if/when the function completes the generator will raise a `StopIteration` exception (just like an iterator).

### Generators as Data Structure Iterators

In [118]:
class ArrayList:
    def __init__(self):
        self.data = np.empty(1, dtype=object)
        self.size = 0
        
    def append(self, val):
        if self.size == len(self.data):
            ndata = np.empty(len(self.data)*2, dtype=object)
            for i in range(len(self.data)):
                ndata[i] = self.data[i]
            self.data = ndata

        self.data[self.size] = val
        self.size += 1
        
    def __iter__(self):
        for i in range(self.size):
            yield self.data[i]

In [119]:
l = ArrayList()
for x in range(10):
    l.append(2**x)

In [120]:
for x in l:
    print(x)

1
2
4
8
16
32
64
128
256
512


In [121]:
# use the now functioning iterator to implement __repr__
class ArrayList(ArrayList):
    def __repr__(self):
        return '[' + ', '.join(repr(x) for x in self) + ']'

In [122]:
l = ArrayList()
for x in range(10):
    l.append(2**x)
l

[1, 2, 4, 8, 16, 32, 64, 128, 256, 512]